## Obtain gnomAD variants corresponding to the 65 CH genes

### CH gene exons coordenates

Corresponding to: "CH_genes_exons_Clust.tsv"

Obtained by notebook: **"CH_genes_exons"**


### Variant data

Obtained from: "/workspace/datasets/gnomad/data/v2.1.1/exomes/hg38/gnomad.exomes.r2.1.1.sites.liftover_grch38.vcf.bgz"

gnomAD v2.1.1, taking only exons:

- total: 125,748 samples

- non-cancer: 118,479 samples

- controls/biobank: 54,704 samples


In [1]:
import pysam
import pandas as pd

In [2]:
### GET ALL VARIANTS FROM ALL CH GENES EXONS

### Open CH genes exon coordenates
# Open file
f = open("/home/sdemajo/CH_gnomad_2021_02/results/CH_genes_exons_Clust.tsv")
# Read lines skipping the first
ch_exons = f.readlines()[1:]
# Close file
f.close()

### Creat tabix object with TabixFile
# (allows fast random access to compressed and tabix indexed tab-separated file formats with genomic data)
file = "/workspace/datasets/gnomad/data/v2.1.1/exomes/hg38/gnomad.exomes.r2.1.1.sites.liftover_grch38.vcf.bgz"
tbx = pysam.TabixFile(file)

### Obtain variant data

# Create an empty list for all variants
variant_all = []

# Iterate every exon position
for n, line in enumerate(ch_exons):
    # Split data tab
    exon_data = line.split("\t")
    # Obtain localization and gene info for every exon
    chrom = "chr" + exon_data[1]
    start = int(exon_data[2])
    end = int(exon_data[3])
    gene = exon_data[7].replace('\n','')
    transcript = exon_data[6].replace('\n','')

    # Create an empty list for variants from this exon
    variant_list = []

    # Fetch to iterate over all of the read mapping to a specified region
    for row in tbx.fetch(chrom, start, end):

        # get variant data (except last column)
        var_info = row.split("\t")[0:-1]

        # add gene name and transcript code
        var_info.append(gene)
        var_info.append(transcript)

        # get AF: allel frequency
        var_info2 = row.split("\t")[-1] # get last column (INFO)
        var_info2 = var_info2.split(";") # split by ;
        # iterate through INFO data and extract AF
        for i in var_info2:
            if i[0:3] == "AF=":
                var_info.append(i)
            elif i[0:12] == "controls_AF=":
                var_info.append(i)
            elif i[0:14] == "non_cancer_AF=":
                var_info.append(i)

        # Add variant to the exon list
        variant_list.append(var_info)

    # Add 
    variant_all.append(variant_list)




In [3]:
### Check results
print(type(variant_all))
print(len(variant_all))

<class 'list'>
1146


In [4]:
### TRANSFORM RESULTS TO DATA FRAME

## 1. Transform list of lists to one list

ch_variants = [item for sublist in variant_all for item in sublist]


## 2. Transform list (of lists) to data frame
ch_variants = pd.DataFrame(ch_variants, columns = ["Chromosome", "Position", "rsID", "REF", "ALT", "QUAL", "FILTER",
                                         "Gene", "Transcript", "AF", "controls_AF", "non_cancer_AF"])
ch_variants

,Chromosome,Position,rsID,REF,ALT,QUAL,FILTER,Gene,Transcript,AF,controls_AF,non_cancer_AF
0,chr1,1787332,rs747120803,T,C,4570.98,PASS,GNB1,ENST00000378609,AF=1.59937e-05,controls_AF=9.18560e-06,non_cancer_AF=1.69804e-05
1,chr1,1787334,rs1362537090,G,A,314.39,PASS,GNB1,ENST00000378609,AF=3.99505e-06,controls_AF=9.17852e-06,non_cancer_AF=4.24128e-06
2,chr1,1787357,rs1386056041,C,T,372.36,AC0;RF,GNB1,ENST00000378609,AF=0.00000e+00,controls_AF=0.00000e+00,non_cancer_AF=0.00000e+00
3,chr1,1787367,rs750998849,T,C,1667.37,PASS,GNB1,ENST00000378609,AF=3.98238e-06,controls_AF=0.00000e+00,non_cancer_AF=4.22697e-06
4,chr1,1787370,rs139152573,C,T,7962.49,PASS,GNB1,ENST00000378609,AF=3.58403e-05,controls_AF=1.83046e-05,non_cancer_AF=2.95878e-05
...,...,...,...,...,...,...,...,...,...,...,...,...
54150,chrX,124095410,rs750598294,G,A,1182.36,PASS,STAG2,ENST00000371145,AF=5.45461e-06,controls_AF=1.24504e-05,non_cancer_AF=5.80966e-06
54151,chrX,124095442,rs1230398678,A,T,3145.36,PASS,STAG2,ENST00000371145,AF=5.46033e-06,controls_AF=0.00000e+00,non_cancer_AF=5.81564e-06
54152,chrX,124095446,rs201262699,A,T,53165.35,PASS,STAG2,ENST00000371145,AF=1.80286e-04,controls_AF=1.99414e-04,non_cancer_AF=1.57104e-04
54153,chrX,124100577,rs1304348952,C,A,1266.82,PASS,STAG2,ENST00000371145,AF=5.53710e-06,controls_AF=1.26076e-05,non_cancer_AF=5.90099e-06


In [5]:
### MODIFY DATA FRAME

# Eliminate "AF=" text
ch_variants["AF"] = ch_variants["AF"].str.replace("AF=", "")
ch_variants["controls_AF"] = ch_variants["controls_AF"].str.replace("controls_AF=", "")
ch_variants["non_cancer_AF"] = ch_variants["non_cancer_AF"].str.replace("non_cancer_AF=", "")

# Transform to number
ch_variants["AF"] = pd.to_numeric(ch_variants["AF"])
ch_variants["controls_AF"] = pd.to_numeric(ch_variants["controls_AF"])
ch_variants["non_cancer_AF"] = pd.to_numeric(ch_variants["non_cancer_AF"])

print(ch_variants.dtypes)
ch_variants

Chromosome        object
Position          object
rsID              object
REF               object
ALT               object
QUAL              object
FILTER            object
Gene              object
Transcript        object
AF               float64
controls_AF      float64
non_cancer_AF    float64
dtype: object


,Chromosome,Position,rsID,REF,ALT,QUAL,FILTER,Gene,Transcript,AF,controls_AF,non_cancer_AF
0,chr1,1787332,rs747120803,T,C,4570.98,PASS,GNB1,ENST00000378609,0.000016,0.000009,0.000017
1,chr1,1787334,rs1362537090,G,A,314.39,PASS,GNB1,ENST00000378609,0.000004,0.000009,0.000004
2,chr1,1787357,rs1386056041,C,T,372.36,AC0;RF,GNB1,ENST00000378609,0.000000,0.000000,0.000000
3,chr1,1787367,rs750998849,T,C,1667.37,PASS,GNB1,ENST00000378609,0.000004,0.000000,0.000004
4,chr1,1787370,rs139152573,C,T,7962.49,PASS,GNB1,ENST00000378609,0.000036,0.000018,0.000030
...,...,...,...,...,...,...,...,...,...,...,...,...
54150,chrX,124095410,rs750598294,G,A,1182.36,PASS,STAG2,ENST00000371145,0.000005,0.000012,0.000006
54151,chrX,124095442,rs1230398678,A,T,3145.36,PASS,STAG2,ENST00000371145,0.000005,0.000000,0.000006
54152,chrX,124095446,rs201262699,A,T,53165.35,PASS,STAG2,ENST00000371145,0.000180,0.000199,0.000157
54153,chrX,124100577,rs1304348952,C,A,1266.82,PASS,STAG2,ENST00000371145,0.000006,0.000013,0.000006


In [6]:
### FILTER VARIANTS THAT DO NOT PASS QC

# Check FILTER variable
print(ch_variants["FILTER"].value_counts())
print()

# Select only good QC
ch_variants = ch_variants[ch_variants["FILTER"] == "PASS"]
print(ch_variants["FILTER"].value_counts()) # Check

PASS                      47956
AC0;RF                     2587
AC0                        1861
RF                         1721
InbreedingCoeff;RF           14
InbreedingCoeff              13
AC0;InbreedingCoeff           2
AC0;InbreedingCoeff;RF        1
Name: FILTER, dtype: int64

PASS    47956
Name: FILTER, dtype: int64


In [7]:
### SAVE

ch_variants.to_csv("/home/sdemajo/CH_gnomad_2021_02/results/CH_genes_variants.tsv", sep="\t")

ch_variants

,Chromosome,Position,rsID,REF,ALT,QUAL,FILTER,Gene,Transcript,AF,controls_AF,non_cancer_AF
0,chr1,1787332,rs747120803,T,C,4570.98,PASS,GNB1,ENST00000378609,0.000016,0.000009,0.000017
1,chr1,1787334,rs1362537090,G,A,314.39,PASS,GNB1,ENST00000378609,0.000004,0.000009,0.000004
3,chr1,1787367,rs750998849,T,C,1667.37,PASS,GNB1,ENST00000378609,0.000004,0.000000,0.000004
4,chr1,1787370,rs139152573,C,T,7962.49,PASS,GNB1,ENST00000378609,0.000036,0.000018,0.000030
6,chr1,1787388,rs766891832,G,A,2299.37,PASS,GNB1,ENST00000378609,0.000004,0.000000,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...
54150,chrX,124095410,rs750598294,G,A,1182.36,PASS,STAG2,ENST00000371145,0.000005,0.000012,0.000006
54151,chrX,124095442,rs1230398678,A,T,3145.36,PASS,STAG2,ENST00000371145,0.000005,0.000000,0.000006
54152,chrX,124095446,rs201262699,A,T,53165.35,PASS,STAG2,ENST00000371145,0.000180,0.000199,0.000157
54153,chrX,124100577,rs1304348952,C,A,1266.82,PASS,STAG2,ENST00000371145,0.000006,0.000013,0.000006


In [8]:
### STUDY RESULTS

## 1. Number of variants for each gene
# (Non-normalized by size)
ch_variants["Gene"].value_counts()

KMT2D     4141
KMT2C     3272
NOTCH1    2296
ATM       2211
APC       2090
          ... 
GNB1       158
SDHAF2     142
U2AF1      106
NRAS        99
KRAS        72
Name: Gene, Length: 65, dtype: int64

In [9]:
## 2. Maximum allel frequency each gene
ch_variants.sort_values(by = ["AF"], ascending=False).head(20)

,Chromosome,Position,rsID,REF,ALT,QUAL,FILTER,Gene,Transcript,AF,controls_AF,non_cancer_AF
21631,chr17,76736425,rs237058,A,G,446471437.84,PASS,SRSF2,ENST00000392485,0.997716,0.997639,0.997741
17209,chr15,52397434,rs1724577,T,G,686821345.29,PASS,MYO5A,ENST00000399231,0.968857,0.965944,0.968792
16892,chr15,52375355,rs2414145,G,A,660859405.83,PASS,MYO5A,ENST00000399231,0.959533,0.955890,0.959584
3400,chr10,121913824,rs10749435,T,C,300183430.26,PASS,ATE1,ENST00000224652,0.899615,0.894335,0.898484
3158,chr10,121836739,rs4237536,T,C,359903111.56,PASS,ATE1,ENST00000224652,0.895802,0.891226,0.894579
21695,chr17,76737017,rs237057,G,A,374166392.60,PASS,SRSF2,ENST00000392485,0.894510,0.890979,0.894797
14484,chr15,41699117,rs2178004,A,T,247403229.11,PASS,MGA,ENST00000219905,0.822932,0.822809,0.824158
14881,chr15,41734566,rs1918314,C,T,174505923.81,PASS,MGA,ENST00000219905,0.800069,0.798792,0.800607
39439,chr5,112841059,rs459552,T,A,493715924.54,PASS,APC,ENST00000257430,0.794825,0.793104,0.794920
15384,chr15,41749514,rs2695163,G,A,357988193.93,PASS,MGA,ENST00000219905,0.792394,0.790177,0.793146


In [10]:
## 3. Mean AF per gene
ch_variants.groupby(["Gene"])["AF"].mean().sort_values(ascending=False)

Gene
SRSF2     0.011968
ATE1      0.006188
SF3B1     0.005159
RET       0.002896
ZRSR2     0.002461
            ...   
MYCN      0.000040
MYD88     0.000040
KRAS      0.000032
SDHAF2    0.000027
NRAS      0.000015
Name: AF, Length: 65, dtype: float64